# **Model Specific Methods**

## **Linear Regression**

To get started, we will revisit our old friend, linear regression. You have likely used this model and even explained its output before. But there is a reason linear regression is so often used—in addition to being easy to train, it’s also easy to understand. So, we will start with a quick review of what makes it a white-box model and how to explain its predictions. Fortunately, linear regression models are fully interpretable, and we can examine how each input feature affects the output. Linear regression models are represented using a linear equation and defined by their coefficients. Here, the coefficients are a model-specific interpretation technique and do not apply to other types of models.

### **Coefficients & Intercepts**

The linear regression equation is determined from the coefficient and intercept values.

- **Unstandardized Coefficients**: These are the coefficients directly estimated by the regression model. For a continuous independent variable, the unstandardized coefficient represents the change in the dependent variable for a one-unit increase in the independent variable, ceteris paribus (all else being equal). For example, if the coefficient for "Years of Experience" in a salary prediction model is 1500, it means that a one-year increase in experience is associated with a £1500 increase in salary, holding other factors (education, job title, etc.) constant.
- **Standardized Coefficients (Beta Coefficients)**: These coefficients are standardized to have a mean of 0 and a standard deviation of 1. This allows for a direct comparison of the relative importance of different independent variables, regardless of their units of measurement. A larger absolute value of the standardized coefficient indicates a stronger effect. However, standardized coefficients are less intuitive to interpret in terms of the original units of measurement. Each coefficient represents the expected change in the target variable (bike rentals) for a one standard deviation increase in the corresponding feature, holding all other features constant. For example, one standard deviation increase in temperature increases the number of daily bike rentals increases by 1195. On the other hand, a one-standard-deviation increase in windspeed decreases the number of daily bike rentals by 321 (since the sign is negative).
- **Dummy Variables (two categories)**: Categorical variables are typically represented using dummy variables (0/1). One category is chosen as the reference category (coded as 0), and the coefficients for the other categories represent the difference in the dependent variable compared to the reference category, holding other variables constant. For example, if we have a categorical variable "Gender" with categories "Male" (reference) and "Female", and the coefficient for "Female" is 5000 in a salary model, it means that women earn £5000 more than men on average, controlling for other factors.
- **Dummy Variables (multiple categories)**: With more than two categories, you'll have multiple dummy variables. Each coefficient represents the difference from the reference category.
- **Intercept**: The intercept represents the baseline prediction of bike rentals when all the features are at their mean values; since the `hum_s`, `temp_s`, `windspeed_s` are standardized, their mean is 0. This means that when all features are at their average values, the expected number of bike rentals is 4,561, the intercept value (which is also the same as the average bike rentals in the training set)
- **Statistical Significance**: The interpretation of coefficients is only meaningful if they are statistically significant (p-value < significance level, usually 0.05). A non-significant coefficient suggests that the effect of the corresponding independent variable is not reliably different from zero.
- **Model Assumptions**: The validity of the coefficient interpretations relies on the satisfaction of the linear regression assumptions (linearity, independence of errors, homoscedasticity, normality of errors). Violations of these assumptions can lead to biased or unreliable coefficient estimates.


## **Logistic Regression**

The logistic regression model estimates the log-odds of the outcome (usually coded as 1) as a linear function of the predictor variables:

$\log(\frac{p}{1-p}) = \beta_0 + \beta_1X_1 + \beta_2X_2 + ... + \beta_kX_k$

Where:

- $p$ is the probability of the outcome (dependent variable) being 1.
- $\frac{p}{1-p}$ is the odds of the outcome being 1.
- $\beta_0$ is the intercept (log-odds when all predictors are 0).


### **Coefficients**

Interpreting these coefficients requires understanding three key points:
- **Direction**: The sign of each coefficient indicates whether the feature increases (positive) or decreases (negative) the log-odds of high rentals. In this case, all features have a positive relationship with high rentals.
- **Magnitude**: The coefficients represent the change in log odds for a one-standard-deviation increase in the feature, holding other features constant. For example, a one-standard-deviation increase in temperature increases the log odds of high rentals by 1.883.

### **Odds Ratios**

Interpreting coefficients in logistic regression is different from linear regression because the dependent variable is a probability (between 0 and 1), not a continuous variable. The coefficients represent the change in the log-odds of the outcome for a one-unit change in the predictor variable, holding other variables constant. To make the interpretation more intuitive, we often transform the coefficients into odds ratios. To obtain a more interpretable measure, we exponentiate the coefficient:

$\frac{p}{1-p} = e^{\beta_0 + \beta_1X_1 + \beta_2X_2 + ... + \beta_kX_k}$

$\frac{p}{1-p} = e^{\beta_0}e^{\beta_1X_1}e^{\beta_2X_2}e^{\beta_kX_k}$

This will have a multiplicative effect (not additive) on the odds. 

```python
import numpy as np

for coef, feature in zip(log_reg.coef_[0], features):
    odds_ratio = np.exp(coef)
    print(f'{feature}: {odds_ratio:.2f}x odds')

```
Example odds ratio interpretation:

- If odds ratio = 2.0: Doubling of odds for one unit increase in feature
- If odds ratio = 0.5: Halving of odds for one unit increase in feature
- If odds ratio = 1.0: No change in odds

While odds ratios are helpful, often, we want to know how probabilities change. However, unlike linear regression, the change in likelihood for a unit change in a feature depends on the current probability. This is due to the logistic function’s S-shaped curve and nonlinear nature.

For example, when we’re at P = 0.5 (logit = 0):
```python
def prob_change_at_mean(coefficient):
    return 1/(1 + np.exp(-coefficient)) - 0.5

for coef, feature in zip(log_reg.coef_[0], features):
    delta_p = prob_change_at_mean(coef)
    print(f'{feature}: {delta_p:.3f} change in probability')
```

Example interpretations:

- Change = 0.2: A one-unit increase in the feature increases the probability by 20 percentage points
- Change = -0.1: A one-unit increase in the feature decreases probability by 10 percentage points
- Change = 0.0: Feature has no effect on probability at the mean

## **Decision Trees (Regression)**

Unlike linear regression, which assumes a linear relationship between features and the target variable, decision trees split the feature space into regions, which allows them to capture non-linear relationships. 

### **Feature Importance (MSE)**

Unlike linear regression, which uses coefficients to explain feature importance, decision trees determine feature importance based on how much each feature reduces the impurity of the data at each split. The more a feature reduces impurity, the higher its importance. The default impurity measure for regression trees is mean squared error (MSE). This can be changed using the parameter `criterion="squared_error"` (or other measures) in `DecisionTreeRegressor()`.

- At each split, the model calculates how much a feature reduces the MSE in the predicted target (e.g., daily bike rentals).
- The overall importance of a feature is computed as the weighted average reduction in MSE across all the splits where it was used.
- Features that consistently result in the most significant reduction in MSE across multiple splits are assigned higher importance.

We can also plot the decision tree structure to understand how it makes predictions. The root node represents the first and most crucial split.

## **Decision Trees (Classification)**

Unlike regression trees, which use MSE, classification trees typically use Gini impurity to measure the quality of splits.

### **Feature Importance (Gini Impurity)**

For a binary classification problem, Gini impurity is calculated as:

$\text{Gini} = 1 - p_1^2 - p_2^2$

where $p_1$ and $p_2$ are the probabilities of each class (for high/low rentals).

For example, if a node has:
- 100% samples of one class: Gini = 1 - (1² + 0²) = 0 (pure node)
- 50% samples of each class: Gini = 1 - (0.5² + 0.5²) = 0.5 (maximum impurity for binary classification)

The default measure in Scikit-learn is Gini impurity and can be changed using the parameter `criterion="gini"` (or other measures) in `DecisionTreeClassifier()`.
- At each split, the model calculates how much a feature reduces the Gini impurity in the predicted target (e.g., class labels).
- A feature’s overall importance is determined by the weighted average reduction in Gini impurity across all the splits where it appears.

Feature importance is still computed based on impurity reduction, but now, Gini impurity is used instead of MSE. Feature importances are computed using the `feature_importances_` attribute of the model.

## **Ensemble Methods**

In ensemble tree methods like Random Forest and Gradient Boosting, feature importance is calculated by measuring how much each feature contributes to reducing impurity (e.g., Gini impurity for classification, MSE for regression) across all trees in the model. At each split, the reduction in impurity is recorded for the chosen feature, and the total reduction is averaged and weighted by the number of times the feature is used. Features that lead to greater reductions in impurity across multiple trees receive higher importance scores, helping identify the most influential predictors in the dataset. Feature importance is a tree-specific explainability technique, but it applies to other tree-based models, not just single decision trees.